In [9]:
import pandas as pd
import re
!pwd
dataset = pd.read_csv('/Users/nikhil/pre_processed_patient_diagnostics.csv', encoding='ISO-8859-1');

dataset.columns

/Users/nikhil


Index(['Unnamed: 0', 'Case', 'Age', 'Text Details', 'Surgery - Yes OR No',
       'word_count'],
      dtype='object')

In [10]:
dataset['Surgery - Yes OR No'] = dataset['Surgery - Yes OR No'].astype(str) # taking the target column in seperate variable. we will add this to final data frame when pre processing is done
data_desc = dataset["Text Details"] 

In [11]:
for i in range(1,len(data_desc)):
    data_desc[i] = re.sub(" \d+", " ",data_desc[i]) # this will retain only the words

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [13]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer(max_features=1000, lowercase=True, ngram_range=(1,1),analyzer = "word")
data_mat = bow.fit_transform(data_desc)
dense = data_mat.todense()

print(dense)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [20]:
# extracting the features and storing it in a different dataset
featureset = pd.DataFrame(dense,columns=bow.get_feature_names())
featureset["Surgery - Yes OR No"] = dataset['Surgery - Yes OR No']
classlabel = "Surgery - Yes OR No"
featureset["Age"]=dataset["Age"]

print(featureset)


     abdomen  abdominal  abdominalback  ablation  abnormality  above  \
0          0          0              0         0            1      0   
1          0          0              0         0            0      1   
2          0          0              0         0            0      0   
3          1          0              0         0            0      0   
4          0          0              0         0            0      0   
5          0          0              0         0            0      0   
6          0          0              0         0            0      0   
7          0          0              0         0            0      0   
8          1          0              0         0            0      0   
9          0          0              0         0            0      0   
10         0          0              0         0            0      0   
11         0          0              0         0            0      0   
12         1          0              0         0            0   

In [21]:
# shuffing the data to avoid baising 
import numpy as np
featureset = featureset.sample(frac = 1)
print(featureset.shape)
Targetindex = featureset.columns.get_loc("Surgery - Yes OR No")
np.random.seed(0)

(117, 1002)


In [22]:
def Average(lst):
    return sum(lst) / len(lst)

In [48]:
#Test split out of training (Divided training into test and train sets respectively)
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.cross_validation import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
def splitter(trials,data,size):
    
    model_acc=[]
    model_f1=[]
    model_recall=[]
    model_precision=[]
    
  
    for i in range(0,trials):
        dataset_train, dataset_test = train_test_split(data, test_size=size)
        dataset_train = pd.DataFrame(dataset_train)
        dataset_test = pd.DataFrame(dataset_test)
        dataset_train.columns = featureset.columns
        dataset_test.columns = featureset.columns
        Targetindex = dataset_train.columns.get_loc("Surgery - Yes OR No")
        
        X = dataset_train.drop(dataset_train.columns[[Targetindex]], axis=1) # all the features except the target variable
        y = dataset_train["Surgery - Yes OR No"] # only the target variable
        X1 = dataset_test.drop(dataset_test.columns[[Targetindex]], axis=1) # all the features except the target variable
        y1 = dataset_test["Surgery - Yes OR No"] # only the target variable
        
        # label encode the target variable 
        encoder = preprocessing.LabelEncoder()
        #print(y)
        train_y = encoder.fit_transform(y)
        #print(train_y)
        valid_y = encoder.fit_transform(y1)
        
        clf = linear_model.LogisticRegression().fit(X, train_y)
        
        testpred = clf.predict(X1)
        print(clf.coef_.shape)
        model_acc.append(accuracy_score(valid_y,testpred))
        model_f1.append(f1_score(valid_y,testpred,pos_label=1,average="binary"))
        model_recall.append(recall_score(valid_y,testpred,pos_label=1,average="binary"))
        model_precision.append(precision_score(valid_y,testpred,pos_label=1,average="binary"))
        
        matrix=confusion_matrix(valid_y,testpred)
        print("Confusion matrix for split",i)
        print(matrix)
        
    print("Average metrics for Logistic Regression are:")
    
    acc_avg=Average(model_acc)
    f1_avg=Average(model_f1)
    recall_avg=Average(model_recall)
    precision_avg=Average(model_precision)

    return acc_avg,f1_avg,recall_avg,precision_avg




In [49]:
splitter(10,np.array(featureset),0.3)


(1, 1001)
Confusion matrix for split 0
[[ 0  2]
 [ 1 33]]
(1, 1001)
Confusion matrix for split 1
[[ 2  1]
 [ 0 33]]
(1, 1001)
Confusion matrix for split 2
[[ 2  1]
 [ 2 31]]
(1, 1001)
Confusion matrix for split 3
[[ 1  2]
 [ 1 32]]
(1, 1001)
Confusion matrix for split 4
[[ 2  2]
 [ 1 31]]
(1, 1001)
Confusion matrix for split 5
[[ 2  6]
 [ 0 28]]
(1, 1001)
Confusion matrix for split 6
[[ 2  4]
 [ 0 30]]
(1, 1001)
Confusion matrix for split 7
[[ 0  6]
 [ 1 29]]
(1, 1001)
Confusion matrix for split 8
[[ 1  7]
 [ 0 28]]
(1, 1001)
Confusion matrix for split 9
[[ 2  3]
 [ 0 31]]
Average metrics for Logistic Regression are:


(0.8888888888888887,
 0.9380281095779777,
 0.9815095811051695,
 0.9008984275528393)

In [65]:
# Bag of Words Model using L1 and L2 Regularization
#Test split out of training (Divided training into test and train sets respectively)

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.cross_validation import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
def splitter1(trials,data,size):
    
    model_acc=[]
    model_f1=[]
    model_recall=[]
    model_precision=[]
    
  
    for i in range(0,trials):
        dataset_train, dataset_test = train_test_split(data, test_size=size)
        dataset_train = pd.DataFrame(dataset_train)
        dataset_test = pd.DataFrame(dataset_test)
        dataset_train.columns = featureset.columns
        dataset_test.columns = featureset.columns
        Targetindex = dataset_train.columns.get_loc("Surgery - Yes OR No")
        
        X = dataset_train.drop(dataset_train.columns[[Targetindex]], axis=1) # all the features except the target variable
        y = dataset_train["Surgery - Yes OR No"] # only the target variable
        X1 = dataset_test.drop(dataset_test.columns[[Targetindex]], axis=1) # all the features except the target variable
        y1 = dataset_test["Surgery - Yes OR No"] # only the target variable
        
        # label encode the target variable 
        encoder = preprocessing.LabelEncoder()
        #print(y)
        train_y = encoder.fit_transform(y)
        #print(train_y)
        valid_y = encoder.fit_transform(y1)
        #Regularization penalty
        clf = linear_model.LogisticRegression(penalty='l2', tol=1e-6,C=100).fit(X, train_y)
        
        testpred = clf.predict(X1)
        print(clf.coef_.shape)
        model_acc.append(accuracy_score(valid_y,testpred))
        model_f1.append(f1_score(valid_y,testpred,pos_label=1,average="binary"))
        model_recall.append(recall_score(valid_y,testpred,pos_label=1,average="binary"))
        model_precision.append(precision_score(valid_y,testpred,pos_label=1,average="binary"))
        
        matrix=confusion_matrix(valid_y,testpred)
        print("Confusion matrix for split",i)
        print(matrix)
        
    print("Average metrics for Logistic Regression are:")
    
    acc_avg=Average(model_acc)
    f1_avg=Average(model_f1)
    recall_avg=Average(model_recall)
    precision_avg=Average(model_precision)

    return acc_avg,f1_avg,recall_avg,precision_avg





In [58]:
splitter1(10,np.array(featureset),0.3)

(1, 1001)
Confusion matrix for split 0
[[ 1  2]
 [ 3 30]]
(1, 1001)
Confusion matrix for split 1
[[ 0  8]
 [ 0 28]]
(1, 1001)
Confusion matrix for split 2
[[ 1  3]
 [ 0 32]]
(1, 1001)
Confusion matrix for split 3
[[ 2  2]
 [ 0 32]]
(1, 1001)
Confusion matrix for split 4
[[ 1  3]
 [ 1 31]]
(1, 1001)
Confusion matrix for split 5
[[ 1  4]
 [ 0 31]]
(1, 1001)
Confusion matrix for split 6
[[ 0  4]
 [ 2 30]]
(1, 1001)
Confusion matrix for split 7
[[ 2  7]
 [ 0 27]]
(1, 1001)
Confusion matrix for split 8
[[ 2  7]
 [ 1 26]]
(1, 1001)
Confusion matrix for split 9
[[ 2  2]
 [ 1 31]]
Average metrics for Logistic Regression are:


(0.861111111111111, 0.9216635283401861, 0.9747053872053872, 0.8771962269756388)

In [68]:
# 10 fold cross validation
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.cross_validation import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold

data =np.array(featureset)

kf = KFold(n_splits=10, random_state=None, shuffle=False)
model_acc=[]
model_f1=[]
model_recall=[]
model_precision=[]
for train_index, test_index in kf.split(data):
    print("TRAIN:", train_index, "TEST:", test_index)
    data_train, data_test = data[train_index], data[test_index]
    data_train = pd.DataFrame(data_train)
    data_test = pd.DataFrame(data_test)
    data_train.columns = featureset.columns
    data_test.columns = featureset.columns
    Targetindex = data_train.columns.get_loc("Surgery - Yes OR No")
    X =data_train.drop(data_train.columns[[Targetindex]], axis=1) # all the features except the target variable
    y = data_train["Surgery - Yes OR No"] # only the target variable
    X1 = data_test.drop(data_test.columns[[Targetindex]], axis=1) # all the features except the target variable
    y1 = data_test["Surgery - Yes OR No"] # only the target variable
        
    # label encode the target variable 
    encoder = preprocessing.LabelEncoder()
    #print(y)
    train_y = encoder.fit_transform(y)
    #print(train_y)
    valid_y = encoder.fit_transform(y1)
    #Regularization penalty
    clf = linear_model.LogisticRegression(penalty='l2', tol=1e-6,C=100).fit(X, train_y)
        
    testpred = clf.predict(X1)
    print(clf.coef_.shape)
    model_acc.append(accuracy_score(valid_y,testpred))
    model_f1.append(f1_score(valid_y,testpred,pos_label=1,average="binary"))
    model_recall.append(recall_score(valid_y,testpred,pos_label=1,average="binary"))
    model_precision.append(precision_score(valid_y,testpred,pos_label=1,average="binary"))
        
    matrix=confusion_matrix(valid_y,testpred)
    
    print(matrix)
        
    
    
acc_avg=Average(model_acc)
f1_avg=Average(model_f1)
recall_avg=Average(model_recall)
precision_avg=Average(model_precision)
print("Average metrics for Logistic Regression are:")
print(acc_avg)
print(f1_avg)
print(recall_avg)
print(precision_avg)

TRAIN: [ 12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29
  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47
  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116] TEST: [ 0  1  2  3  4  5  6  7  8  9 10 11]
(1, 1001)
[[0 2]
 [1 9]]
TRAIN: [  0   1   2   3   4   5   6   7   8   9  10  11  24  25  26  27  28  29
  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47
  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116] TEST: [12 13 14 15 16 17 18 19 20 21 22 23]
(1, 1001)
[[2 1]
 [

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
